1. CIFAR-10 – get to know (10 points)
CIFAR-10 is a benchmark popular datasets used in machine learning literature for bechmarking different
methods. For brief introduction check https://www.cs.toronto.edu/~kriz/cifar.html.
Download the dataset to your local computer (perhaps your prefer /worktemp instead of your home directory) from https://tuni-my.sharepoint.com/:u:/g/personal/joni_kamarainen_tuni_fi/EdgoMFSFCKRGiM3QcSZPqxUBZz-moWenG9HPWp_
tQ9VISA?e=YAz0Vn. Extract the files and then edit configuration paths in the provided cifar10 illustrate.py.
Run the code:
(intro_prml) kamarain@Joni-Precision-5520:~/exercises$ python cifar10_illustrate.py
The code should randomly plot images from the dataset. Note that the code loads only one of the training
batches that each contains 10,000 training samples. Test data is also provided separately (10,000 samples,
1,000 per each class).

In [3]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from random import random
from scipy import stats
from scipy.spatial.distance import cityblock
from math import sqrt

def unpickle(file):
    with open(file, 'rb') as f:
        dict = pickle.load(f, encoding="latin1")
    return dict

"""OPENING ALL THE FILES INTO ONE ARRAY - An Array of 50000 by 3072 """

a= 'D:/cifar-10-python/data_batch_1'
b= 'D:/cifar-10-python/data_batch_2'
c= 'D:/cifar-10-python/data_batch_3'
d= 'D:/cifar-10-python/data_batch_4'
e= 'D:/cifar-10-python/data_batch_5'

#Adding All Files into one Array
datadict = unpickle(a)
X = datadict["data"].astype("int64") #Basic Data Set with 10,000 rows and 3072 columns
Y = datadict["labels"] #contains the number associated with label name

for i in [b,c,d,e]:
    datadict2 = unpickle(i)
    X1 = datadict["data"].astype("int32") #Basic Data Set with 10,000 rows and 3072 columns
    Y1 = datadict["labels"] #contains the number associated with label name

    X = np.vstack((X,X1)) # Vertical Stack
    Y = np.hstack((Y,Y1)) # horizontal Stack
  
"""BRINGING IN THE LABEL DATA"""

#number from labels associates with one of the 10 names in this
labeldict = unpickle('D:/cifar-10-python/batches.meta')
label_names = labeldict["label_names"] 

Z = X #Simplified, just 50,000 rows and 3072 columns

# #Reshaping the data to view it in R,G,B arrays and print the image as well, previous data was just rows of bytes
# X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("int64") 
# Y = np.array(Y)

"""CODE FOR SHOWING THE IMAGES - DISABLED FOR NOW"""

#Plotting Some Images 
# for i in range(16):
#     plt.subplot(4,4,i+1)
#     plt.imshow(X[i])
# plt.show()

# for i in range(X.shape[0]):
#     # Show some images randomly
#     if random() > 0.999:
#         plt.figure(1);
#         plt.clf()
#         plt.imshow(X[i])
#         plt.title(f"Image {i} label={label_names[Y[i]]} (num {Y[i]})")
#         plt.pause(0.5)

"""BRINGING IN THE TEST DATA"""

#Bringing in Test Data
testdict = unpickle('D:/cifar-10-python/test_batch')
testdict.keys()
X_test = testdict['data'].astype("int32")
Y_test = np.array(testdict['labels']) #Must convert to array before working

Z_test = X_test
# #Reshaping the data to view it, previous data was just rows of bytes
# X_test = X_test.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("int64") 
# Y_test = np.array(Y_test)


In [26]:
"""
SPLITTING INTO TEST AND TRAIN DATA (Smaller sets are taken during development phase)
"""
X_train = Z
X_test = Z_test[:200]
Y_train = Y
Y_test = Y_test[:200]

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(50000, 3072)
(200, 3072)
(50000,)
(100,)


2. CIFAR-10 – Evaluation (10 points)
Write a python function class acc(pred,gt) that computes the classification accuracy for predicted labels
pred as compared to the ground truth labels gt (Y values in the provided code). Make sure that your
function works properly by using the true labels of CIFAR-10 test samples that should provide perfect
accuracy.

In [22]:
"""
DEFINING EVALUATION FUNCTION
"""
def class_acc(pred,gt):

    count = np.count_nonzero(gt == pred)

    accuracy = count*100/len(pred)

    #print("Accuracy:",accuracy,"%")
    
    return accuracy


"""
GIVES 100 % ACCURACY WHEN TESTED WITH ITESELF
"""

print("Accuracy:",class_acc(Y,Y),"%")

Accuracy: 100.0 %


3. CIFAR-10 – Random classifier (20 points)
Write a function cifar10 classifier random(x) that returns a random class label for the input data x. Make
a script that inputs all CIFAR-10 test samples to this function and evaluates its classification accuracy
using your evaluation function.

In [23]:
"""
DEFININIG RANDOM CLASSIFIER
"""
def cifar10_classifier_random(X):
    classes = [0,1,2,3,4,5,6,7,8,9]
    Y_pred = []
    for i in range(len(X)):
        Y_pred.append(np.random.choice(classes))
        
    return Y_pred


"""
BASELINE ACCURACY IS APPROX ~ 10 % upon testing
"""
pred = cifar10_classifier_random(X_train)
accuracy = class_acc(pred,Y_train)
print("Baseline Accuracy:",accuracy,"%")

Baseline Accuracy: 9.964 %


![](baseline.png)

4. CIFAR-10 – 1-NN classifier (20 points)
Write a function cifar10 classifier 1nn(x,trdata,trlabels) that finds the best match of the input vector x in
the training set trdata and returns the same label. Make a script that inputs all CIFAR-10 test samples
to this function and evaluate its classification accuracy using your evaluation function.
Make sure your 1NN function works by testing with the training data itself (should provide perfect
accuracy).
Hints: During developing stage you might want to use only a sub-set of the full data to make evaluation
faster.

In [24]:
def cifar10_classifier_1nn(X,trdata,trlabels,k=1):
    """
    X; test dataset
    trdata; training data
    trlabels; 
    k is the number of nearest neighbours to check for
    returns; predicted class number/s for the input image/s
    """
    X_test = X 
    X_train = trdata
    Y_train = trlabels
    
    
    
    Y_pred = [] #Creating a list to store Predicted class for each image in testing set
    
    for i in range(0,len(X_test)):
        
        dist = [] #Creating a list to store euclidean distances and emptying it for next round
        
        for j in range(0,len(X_train)): 
            #loop to measure distance of each test image with all the images of the training images
            
            #euclidian = np.linalg.norm(X_test[i] - X_train[j]) #Euclidean distance
            #manhatten = abs(X_test[i] - X_train[j])
            manhatten = cityblock(X_test[i],X_train[j])
            
            #dist.append(euclidian) #eulidean distance is stored in a list
            dist.append(manhatten)
        
        #this list is sorted such that indexes are given for the images from least distnce to max distance  
        #argsort gives the index of an array elements in the ascending order
        #picking the k-nearest neighbours and taking the most occuring class as prediction
        k_nearest  = np.argsort(dist)[:k]  
        
        #print(k_nearest)              

        classes = Y_train[k_nearest]
        #print(classes)

        most_occuring_class = stats.mode(classes)

        #Y_pred.append(label_names[most_occuring_class[0][0]])
        Y_pred.append(most_occuring_class[0][0])
        
    return Y_pred

In [ ]:
"""
CHECKING ACCURACY OF OUR CLASSIFIER
"""
for i in [1,2,3,4,5]:
    pred = cifar10_classifier_1nn(X_test,X_train,Y_train,k=i)
    y = class_acc(pred,Y_test)
    print("Accuracy",y,"%    @ k=",i)
#     plt.bar(i,y)
#     plt.text(i,y, f"{y}%")



D:\Softwares Installed\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


Accuracy 0.0 %    @ k= 1


"""BELOW IS ACCURACY SCREENSHOT FOR k = 1, 10 and 100"""

![](finalaccuracy.png)

#EXCESS WORKING

In [ ]:
# print(np.linalg.norm(X_test[0] - X_train[1])) #Euclidean distance auto
# print(math.sqrt(sum((X_test[0] - X_train[1])**2))) #Euclidean distance Manual
# print(sqrt(np.dot(X_test[0], X_train[1]))) #Euclidean using dot product

# print(sum(abs(X_test[0] - X_train[1]))) #Manhatten Manual
# print(cityblock(X_test[0],X_train[1])) #Manhatten Auto

In [ ]:
# #Defining a function to show picture
# def show_fig(X,Y,i):
#     plt.figure(figsize=(10,2))
#     name = label_names[Y[i]]
#     plt.imshow(X[i])
#     plt.title(f"Class : {name}\nClass # {Y[i]}\nImage # {i}")
    
# show_fig(X,Y,1000)

In [ ]:
"""
PREDICTION OF A SINGLE IMAGE COMPARED WITH THE TRAINING SET
"""
# from scipy import stats
# diff = []

# for i in range(0,len(X)):
#     euclidian = (np.linalg.norm(X[1000] - X[i])) #Euclidean distance
#     diff.append(euclidian)
# print(diff[:10])
# #min_value = min(diff);
# #min_index = diff.index(min_value)

# k_nearest  = np.argsort(diff)[:1]  #picking the k-nearest neighbours and taking the most occuring class as prediction

# # k=10
# # for i in range(k):
# #     show_fig(X,Y,k_nearest[i])

# classes = Y[k_nearest]
# print("Top k classe:",classes)

# most_occuring_class = scipy.stats.mode(classes)
# print("Most Occuring Class:",most_occuring_class[0][0])

# Y_pred = label_names[most_occuring_class[0][0]]
# print("Predicted Class:", Y_pred)

In [8]:
a = [1,1,2,2,2,3,5,3,1,1,1,1,2,6]
stats.mode(a)

ModeResult(mode=array([1]), count=array([6]))